In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import tokenizers
from multi_tokenizer import MultiTokenizer, partition_list_by_lang
from gpt import GPT, GPTConfig
from tqdm import tqdm
import os
import math

assert torch.cuda.is_available()
#torch.manual_seed(1337)
#if torch.cuda.is_available():
    #torch.cuda.manual_seed(1337)

device= 'cuda' if torch.cuda.is_available() else 'cpu'
assert device == 'cuda', "USE A GPU"
torch.set_default_device(device)

In [2]:
list(partition_list_by_lang([0,1,323,323,1,0,3232,2]))

[(1, [323, 323]), (0, [3232, 2])]

In [21]:
# Settings
run = "shared/8k" # multi_8k
run = "multi_8k"

# Data Loading

## Load Tokenizers

In [22]:

EXPT_NAME = 'expt_1'

shared_tokenizer_small_path = f'./{EXPT_NAME}/shared/8k/tokenizer.json'
multi_tokenizer_english = f'./{EXPT_NAME}/multi_8k/english_tokenizer.json'
multi_tokenizer_spanish = f'./{EXPT_NAME}/multi_8k/spanish_tokenizer.json'

shared_tokenizer_small = tokenizers.Tokenizer.from_file(shared_tokenizer_small_path)
multi_tokenizer_english = tokenizers.Tokenizer.from_file(multi_tokenizer_english)
multi_tokenizer_spanish = tokenizers.Tokenizer.from_file(multi_tokenizer_spanish)

multi_tokenizer = MultiTokenizer([multi_tokenizer_english, multi_tokenizer_spanish])

In [23]:
encoded = multi_tokenizer.encode("[BEGIN_EN]Hello, I like to[BEGIN_ES] comer arroz")
decoded = multi_tokenizer.decode(encoded)
decoded

shared_tokenizer_small.encode("Hello, I like to comer arroz").ids


[2127, 17, 409, 689, 281, 911, 6015]

In [24]:
if run == "multi_8k":
    tokenizer = multi_tokenizer
    vocab_size = 8192
elif run == "shared/8k":
    tokenizer = shared_tokenizer_small
    vocab_size = 8192
else:
    raise ValueError(f"Unknown run: {run}")

In [25]:
config = GPTConfig(
    block_size = 256,
    n_layer = 8,
    n_embd=256,
    n_head=8,
    use_multitokenizer=True,
    vocab_size = vocab_size
)


## Load Dataset

In [26]:
if run == "multi_8k":
    train = torch.load(f'./{EXPT_NAME}/multi_8k/train.pt')
    english_test = torch.load(f'./{EXPT_NAME}/multi_8k/english_test.pt')
    spanish_test = torch.load(f'./{EXPT_NAME}/multi_8k/spanish_test.pt')
    translation_test = torch.load(f'./{EXPT_NAME}/multi_8k/translation_test.pt')
elif run == "shared/8k":
    tokenizer = shared_tokenizer_small
    train = torch.load(f'./{EXPT_NAME}/shared/8k/train.pt')
    english_test = torch.load(f'./{EXPT_NAME}/shared/8k/english_test.pt')
    spanish_test = torch.load(f'./{EXPT_NAME}/shared/8k/spanish_test.pt')
    translation_test = torch.load(f'./{EXPT_NAME}/shared/8k/translation_test.pt')

# shuffle train
train = train[torch.randperm(train.size(0))]

In [29]:
tokenizer.decode(train[0].tolist())

'[BEGIN_ES]. ¡Los dos novios querían jugar con el perro!\n\nEl perro no sabía qué hacer. El perro quería jugar con sus novios. Pero no podía jugar con ambos a la vez.\n\nEl perro se puso triste. El perro no sabía qué hacer. El perro estaba muy, muy triste.\n\nDe repente, el perro tuvo una idea. El perro jugó con un novio durante un rato. Luego jugó con el otro novio. ¡Todos estaban felices![END][BEGIN_ES][PROMPT_ES_STORY]Un niño pequeño estaba jugando en el jardín. Vio algo verde y largo en la hierba. "¡Serpiente!" gritó. La serpiente era rápida. Se arrastró hacia un árbol. El niño se asustó. Corrió a decirle a su mamá.\n\nSu mamá fue al jardín. "¡No hay serpiente!" dijo. El niño miró. ¡La serpiente ya no estaba! El niño estaba triste. Luego, vio algo pequeño y amarillo. "¡Pollito!" gritó.\n\nEl pequeño pollito se movía rápido. Se metió debajo del árbol. El niño se rió. "¡Serpiente!" gritó de nuevo. El pollito salió corriendo. ¡El niño se rió mucho! \n\nSu mamá se rió también. "¡No es 

In [30]:
def load_tokens(filename):    
    return torch.load(filename).to(dtype=torch.long)

class DataLoaderLite:
    def __init__(self, data_dir, B, T, split, shuffle):
        self.B, self.T, self.shuffle = B, T, shuffle                
        
        shards = os.listdir(data_dir)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_dir, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"
    
        print(f"found {len(shards)} shards for split {split}")
        
        self.current_shard, self.current_position = -1, 0
        self.reset()

    def reset(self):
        self.current_shard, self.current_position = (self.current_shard + 1) % len(self.shards), 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        if self.shuffle:
            assert False, 'not implemented'
            start = time.time()
            self.shuffle_tokens()
            print(f"shuffled {self.tokens.shape[0]} tokens in {time.time() - start:.1f}s")

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T 
        # if loading the next batch would be out of bounds, advance to next shard
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.reset()
        return x, y

# Train

In [31]:

torch.set_float32_matmul_precision(config.matmul_precision)
optimizer = model.configure_optimizers(weight_decay=config.weight_decay, learning_rate=config.max_lr, beta1=config.beta_1, beta2=config.beta_2, device_type="cuda")

loss_accum = []
train = train.to(torch.int64)
def do_epoch():
    batch_stride = config.batch_size
    for current_step, batch_start_idx in enumerate(tqdm(range(0, train.shape[0]-batch_stride, batch_stride))):
        batch = train[batch_start_idx:batch_start_idx+batch_stride]
        inputs = batch[:, 0:config.block_size].contiguous()
        targets = batch[:, 1:config.block_size+1].contiguous()
        model(inputs, targets)
        model.train()
        optimizer.zero_grad()
        with torch.autocast(dtype=torch.bfloat16, device_type="cuda"):
            logits, loss = model(inputs, targets)
        loss_accum.append(loss.detach())
        loss.backward()
        optimizer.step()
        if current_step % 100 ==0:
            print(f"step: {current_step}, loss: {loss.item()}")

do_epoch()


# current_step = 0  # Initialize the current step counter

# for inputs, targets in train_dataloader:
#     model.train()
#     optimizer.zero_grad()
#     inputs, targets = inputs.to(device), targets.to(device)
#     with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
#         logits, loss = model(inputs, targets)
#     loss = loss
#     loss_accum.append(loss.detach())
#     loss.backward()
#     norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#     optimizer.step()
    
#     current_step += 1
#     if math.log2(step) % 1 == 0:
#         print(f"Step: {current_step}, Loss: {loss.item()}")


num decayed parameter tensors: 34, with 6,356,992 parameters
num non-decayed parameter tensors: 66, with 23,040 parameters
using fused AdamW: True


  0%|          | 0/12760 [00:00<?, ?it/s]

  0%|          | 3/12760 [00:00<09:25, 22.56it/s]

step: 0, loss: 10.382516860961914


  1%|          | 105/12760 [00:04<09:21, 22.55it/s]

...

 99%|█████████▉| 12603/12760 [09:20<00:07, 22.23it/s]

step: 12600, loss: 1.7326565980911255


100%|█████████▉| 12705/12760 [09:25<00:02, 22.21it/s]

step: 12700, loss: 1.7633440494537354


100%|██████████| 12760/12760 [09:28<00:00, 22.46it/s]


NameError: name 'reload' is not defined